# Librerias

In [4]:
import pandas as pd
import zipfile

In [11]:
zip_file = zipfile.ZipFile(r".\Archive.zip")
datos_model = pd.read_csv(zip_file.open("datos_para_modelizacion.csv"))

In [12]:
datos_model

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_267,var_268,var_269,var_270,var_271,var_272,var_273,var_274,var_275,label
0,9248245,0.0,0,0,0.0,0.0,0.00,0.0,0.0,1,...,0.0,1.0,0,0.0,0.0,0,0.0,0,0.0,0
1,10037840,0.0,0,0,0.0,0.0,0.00,0.0,0.0,1,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0
2,10299029,0.0,0,0,0.0,0.0,24.44,0.0,0.0,0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0
3,10907006,0.0,0,0,0.0,0.0,0.00,0.0,0.0,0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0
4,14080498,0.0,0,0,0.0,0.0,0.00,0.0,0.0,0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96183,3027642699,0.0,0,0,0.0,0.0,0.00,0.0,0.0,0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0
96184,3039901891,0.0,0,0,0.0,0.0,0.00,0.0,0.0,1,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0
96185,3040108846,0.0,0,0,0.0,0.0,0.00,0.0,0.0,0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0
96186,3045015574,0.0,0,0,0.0,0.0,0.00,0.0,0.0,0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0


In [13]:
zip_file = zipfile.ZipFile(r".\Archive.zip")
datos_pred = pd.read_csv(zip_file.open("datos_para_prediccion.csv"))

In [14]:
datos_pred

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_266,var_267,var_268,var_269,var_270,var_271,var_272,var_273,var_274,var_275
0,9248245,0.0,0,0,0.0,0.0,0.0,0.0,0.0,1,...,0.00,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0
1,10907006,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,...,0.00,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0
2,14080498,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,...,0.00,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0
3,29764234,0.0,0,0,0.0,0.0,0.0,0.0,0.0,1,...,0.00,0.0,2.0,0,0.0,0.0,0,0.0,0,0.0
4,32791073,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,...,9.09,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71391,3029101742,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,...,0.00,0.0,2.0,0,0.0,0.0,0,0.0,0,0.0
71392,3033578041,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,...,0.00,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0
71393,3035802671,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,...,0.00,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0
71394,3047197511,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,...,0.00,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0
